<a href="https://colab.research.google.com/github/AR-Nazari/ASR/blob/main/ASR_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Test

In [ ]:
!pip install opensmile
!pip install pydub
!pip install hazm
# !pip install parsivar
!pip install spleeter

In [ ]:
import main

In [ ]:
tst = main.pipeline('uploaded_audio/test.mp4', 'uploaded_audio/')

In [ ]:
for i in tst:
    print(f'{i[0]},  {i[1]}')

# API test

In [ ]:
!pip install fastapi uvicorn pyngrok python-multipart

In [3]:
!ngrok authtoken ####

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
!ngrok config add-authtoken ###

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import shutil
import os

In [2]:
# import numpy as np
# import GNet
# import MyAudio
# import MyText
# from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor

In [3]:
# app = FastAPI()

# AUDIO_UPLOAD_DIRECTORY = "./uploaded_audio"
# os.makedirs(AUDIO_UPLOAD_DIRECTORY, exist_ok=True)

# @app.post('/test_audio/')
# async def test_audio(file: UploadFile = File(...)):

#     saved_file_path = os.path.join(AUDIO_UPLOAD_DIRECTORY, file.filename)

#     with open(saved_file_path, "wb") as buffer:
#         shutil.copyfileobj(file.file, buffer)

#     chunks = MyAudio.pipe(saved_file_path, AUDIO_UPLOAD_DIRECTORY)

#     return {'file read': True,
#             'audio pipeline completion': True,
#             'number of chunks': len(chunks)}

In [6]:
app = FastAPI()

@app.get("/", response_class=HTMLResponse)
async def root():
    html_content = """
    <html>
        <head>
            <title>API Home</title>
        </head>
        <body>
            <h1>Welcome to the API</h1>
            <p>Access the API documentation <a href="/docs">here</a>.</p>
        </body>
    </html>
    """
    return html_content

# Define the response model
class TranscriptionResponse(BaseModel):
    transcription: str
    gender: str

# The directory where uploaded audio files will be saved
AUDIO_UPLOAD_DIRECTORY = "./uploaded_audio"

# Ensure the directories exist
os.makedirs(AUDIO_UPLOAD_DIRECTORY, exist_ok=True)

@app.post("/upload_and_process_audio/")
async def upload_and_process_audio(file: UploadFile = File(...)):
    # Save the uploaded file with its original filename and format
    saved_file_path = os.path.join(AUDIO_UPLOAD_DIRECTORY, file.filename)

    with open(saved_file_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    # Use the pipeline to process the saved audio file
    results = await main.pipeline(saved_file_path, AUDIO_UPLOAD_DIRECTORY)

    # Format the response (transcriptions and gender classifications)
    response = [{"transcription": item[0], "gender": item[1]} for item in results]

    return {"results": response}

In [7]:
from pyngrok import ngrok
import uvicorn
import nest_asyncio

In [ ]:
# Apply the nest_asyncio patch to allow multiple event loops
nest_asyncio.apply()

# Setup a tunnel to the FastAPI server
public_url = ngrok.connect(8000, "http")
print(f"Public URL: {public_url}")

# Run FastAPI with uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000, timeout_keep_alive=1200)

In [7]:
ngrok.get_tunnels()

[]

In [8]:
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)